In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import math
import json
import random

In [ ]:
#funkce na generovani

class Planet:
    def __init__(self, mass: int, position_array: float, velocity_array: float):
        self.mass = mass
        self.position = np.array(position_array)
        self.velocity = np.array(velocity_array)


def load_from_json(filename: str):
    f = open(filename)

    json_dict = json.load(f)

    planets = []
    for value in json_dict.values():
        planets.append(Planet(value['mass'], value['position'], value['velocity']))

    f.close()

    return planets

def generate_random_planets(n):

    planets = []

    for i in range(n):
        mass = random.randint(1e24, 1e30)
        position = [random.randint(-1e12,1e12),random.randint(-1e12,1e12)]
        velocity = [np.random.randint(-1e5,1e5),np.random.randint(-1e5,1e5)]

        planets.append(Planet(mass, position, velocity))

    return planets


In [ ]:
#funkce na fyziku / pohyb planet

G = 6.6743e-11

def calculate_all_distances(planets: list[Planet]):

    distances: list[list[float]] = [[0 for _ in range(len(planets))] 
                                        for _ in range(len(planets))]
    for i in range(len(planets)):
        for j in range(i+1,len(planets)):
            dist = np.linalg.norm(planets[i].position - planets[j].position)

            distances[i][j] = dist
            distances[j][i] = dist

    return distances


def calculate_all_forces(planets: list[Planet]):
    distances = calculate_all_distances(planets)

    forces: list[list[np.array]] = [[0 for _ in range(len(planets))] 
                                        for _ in range(len(planets))]
    

    for i in range(len(planets)):
        for j in range(i+1, len(planets)):
            force = G*((planets[i].mass*planets[j].mass)/((distances[i][j])**2))           
            force_vector = force * (planets[j].position-planets[i].position)/(distances[i][j])

            forces[i][j] = force_vector
            forces[j][i] = -force_vector

    return forces

def calculate_all_accelerations(planets: list[Planet]):
    accels: list[np.array] = [0 for _ in range(len(planets))] 
    forces = calculate_all_forces(planets)                                    
    
    for i in range(len(planets)):
        force = np.sum(forces[i])

        accels[i] = force/planets[i].mass

    return accels

def move_planets(planets: list[Planet],time_interval):
    accels = calculate_all_accelerations(planets)

    for i in range(len(planets)):
        planets[i].velocity += accels[i]*time_interval
        planets[i].position += planets[i].velocity*time_interval

    


In [ ]:
#funkce na animace

def show_animation(planets: list[Planet], size, duration, time_interval):
    fig, axes = plt.subplots()
    axes.set_xlim([-size,size])
    axes.set_ylim([-size,size])
   
    dots, = axes.plot([p.position[0] for p in planets], [p.position[1] for p in planets],'.', )


    def animation_frame(i):
        move_planets(planets,time_interval)
        dots.set_xdata([p.position[0] for p in planets])
        dots.set_ydata([p.position[1] for p in planets])
        return dots
        
    anim = animation.FuncAnimation(fig, func=animation_frame, frames=np.arange(0, duration, time_interval), interval=20)
    plt.show()

def save_animation(planets: list[Planet], size, duration, time_interval):
    fig, axes = plt.subplots()
    axes.set_xlim([-size,size])
    axes.set_ylim([-size,size])
    axes.set_aspect(1)
    

    dots, = axes.plot([p.position[0] for p in planets], [p.position[1] for p in planets],'.', )


    def animation_frame(i):
        move_planets(planets,time_interval)
        dots.set_xdata([p.position[0] for p in planets])
        dots.set_ydata([p.position[1] for p in planets])
        return dots
        
    anim = animation.FuncAnimation(fig, func=animation_frame, frames=np.arange(0, duration*time_interval*50, time_interval), interval=20)
    anim.save('planets.mp4',writer='ffmpeg')




In [ ]:

planets = load_from_json('testdata/test1.json')

show_animation(planets, 5e12, 10, 60*60*24)


In [ ]:
%matplotlib widget
planets = load_from_json('testdata/test2.json')
show_animation(planets, 5e11, 10, 60*60*24)


In [ ]:
%matplotlib widget
planets = generate_random_planets(5)
for p in planets:
    print(p.velocity)
show_animation(planets, 1e12, 10, 60*60*24)